# Домашнее задание "Варианционные автоэнкодеры"

**Автор**: Ермекова Асель

В этом домашнем задании вам предстоит реализовать VAE для датасета картинок MNIST.

Вы научитесь обучать вариационный автоэнкодер (VAE) генерировать новые изображения с нуля. А также сможете управлять генерацией, указывая желаемый класс объекта, и оценивать качество результата с помощью метрики FID.

Это домашнее задание состоит из двух частей:

* **I часть.** Реализовать безусловную генерацию картинок при помощи VAE тренированную на датасете MNIST и оценить качество по метрике FID.
* **II часть.** Реализовать условную генерацию по классу и оценить качество по метрике FID.



     

Установите библиотеку для подсчета FID:

In [ ]:
!pip install pytorch-fid

## **I часть. Unconditional VAE (6 баллов)**

### Библиотеки

In [ ]:
# TODO
# Импортните любые необходимые вам библиотеки

### Датасет.

**Задание**: Скачайте датасет MNIST и подготовьте train dataloader.

In [ ]:
# --- Подготовка данных ---
batch_size = # TODO
transform = # TODO

train_dataset = # TODO
train_loader = # TODO

**Задание**: Для FID сохраните 10k реальных изображений из MNIST test части в папку

In [ ]:
# TODO: Для FID сохраните 10k реальных изображений из MNIST test части в папку


**Задание**: Визуализируйте 5 рандомных сэмплов из тренировочных данных и 5 сэмплов из тестовых данных

In [ ]:
# TODO


### Модель

**Задание**: Реализуйте VAE архитектуру

In [ ]:
# TODO: Реализуйте VAE (безусловный)
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        # Encoder: изображение -> mu, logvar
        # Decoder: z -> изображение

    def encode(self, x):
        # TODO
        pass

    def reparameterize(self, mu, logvar):
        # TODO
        pass

    def decode(self, z):
        # TODO
        pass

    def forward(self, x):
        # TODO
        pass

### Loss

**Задание**: Напишите VAE Loss

In [ ]:
# TODO: Реализуйте функцию потерь VAE
def vae_loss(recon_x, x, mu, logvar):
    # Reconstruction loss: BCE (since output is sigmoid)
    recon_loss = # TODO
    # KL divergence: D_KL(q(z|x) || p(z))
    kl_loss = # TODO
    return recon_loss + kl_loss

### Тренировка

**Задание**: Обучите модель на датасете MNIST.

In [ ]:
latent_dim = 20  # MNIST VAEs often use 20–64
epochs = # TODO
lr = 1e-3
img_size = 28
channels = 1

In [ ]:
# TODO: Обучите модель
model = VAE(latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Обучение...

### Метрика

В этом разделе вам необходимо посчитать метрику FID.

**Что такое FID?**

**FID (Fréchet Inception Distance)** — это метрика качества генеративных моделей для изображений, которая измеряет **расстояние между распределениями признаков реальных и сгенерированных изображений** в пространстве предобученной нейросети (обычно Inception-v3).

Чем **ниже FID**, тем **ближе** сгенерированные изображения к реальным — как по **качеству**, так и по **разнообразию**.

Формула FID основана на предположении, что признаки в этом пространстве приблизительно распределены как **многомерное нормальное распределение**:

$$
\text{FID} = \|\mu_r - \mu_g\|^2 + \mathrm{Tr}\left( \Sigma_r + \Sigma_g - 2\sqrt{\Sigma_r \Sigma_g} \right)
$$

где:
- $(\mu_r, \Sigma_r)$ — среднее и ковариационная матрица признаков **реальных** изображений,
- $(\mu_g, \Sigma_g)$ — то же для **сгенерированных** изображений,
- $\mathrm{Tr}(\cdot)$ — след матрицы.

> 🔹 FID = 0 означает полное совпадение распределений.  
> 🔹 Чем выше FID ↑ , тем качество или разнообразие генерации ниже ↓.

**Как считать FID на MNIST?**

Вычислите FID с помощью библиотеки [`pytorch-fid`](https://github.com/mseitzer/pytorch-fid):

```bash
python -m pytorch_fid real_mnist/ fake_mnist/ --device cuda
```

> **Важно**: несмотря на то, что признаки Inception-v3 не оптимальны для рукописных цифр, FID остаётся полезной **относительной метрикой** — она позволяет сравнивать разные модели между собой при одинаковых условиях предобработки.


**Задание:** Сгенерируйте и сохраните 10 тыс. изображений, выберите 10 тыс. реальных изображений из MNIST тестовой выборки и посчитайте FID между реальными и сгенерированными изображениями.

In [ ]:
# TODO: Сгенерируйте и сохраните 10 тыс. изображений для FID в папке mnist_vae_fake


In [ ]:
# Чтобы вычислить FID, запустите в терминале:
!pip install pytorch-fid
!python -m pytorch_fid mnist_vae_real mnist_vae_fake --device cuda

## **II Часть. Conditional VAE (6 баллов)**


Мы уже научились обучать обычный VAE на датасете картинок и получать новые картинки. Давайте теперь научимся обучать модель, которая сможет генерировать не просто рандомную картинку, которая похожа на картинки из датасета, а картинку из конкретного класса. Например, в MNIST датасете 10 классов (от 1 до 10) и мы хотим говорить модели "Сгенерируй мне только конкретно картинку с числом 3." и она должна теперь уже сгенерировать только картинку с числом 3. Как раз Conditional VAE это должен уметь делать и генерировать картинку, обуславливаясь на конкретный класс.


**Задание**. В этой части домашнего задания вам предстоит обучить Conditional VAE на MNIST. Это значит, что модель на вход должна принимать картинку и класс картинки.

**Метрика**. Вам нужно сгенерировать 1000 сэмплов на каждый класс и посчитать FID для каждого класса.

In [ ]:
# TODO: Реализуйте Condiional VAE — добавьте one-hot класс как вход в encoder и decoder
class CVAE(nn.Module):
    def __init__(self, latent_dim, num_classes=10):
        super(CVAE, self).__init__()
        # TODO

    def encode(self, x, c):
        # TODO: конкатенируйте x и c по каналам
        pass

    def decode(self, z, c):
        # TODO: конкатенируйте z и c
        pass

    def forward(self, x, c):
        # TODO
        pass

In [ ]:
# TODO: Обучите CVAE


In [ ]:
# TODO: Сгенерируйте 1000 сэмплов для каждого класса при помощи CVAE модели


# TODO: Сохраните 1000 сэмплов для каждого класса из реального датасета MNIST тестовой части


In [ ]:
# TODO: Посчитайте FID для каждого класса между сгенерированными и реальными изображениями
# Example:
print("Class 0")
!python -m pytorch_fid real_per_class/class_0 fake_per_class/class_0 --device cuda